# Examples for TA2 Model Representation
## Create ASKENET model by hand to demonstrate issues with modeling representation

In [1]:
import sys
sys.path.append('/Users/mahs128/Repos/mira')

In [39]:
sys.path

['/Users/mahs128/Repos/pyciemss/notebook/Examples_for_TA2_Model_Representation',
 '/Users/mahs128/opt/anaconda3/envs/pyciemss/lib/python311.zip',
 '/Users/mahs128/opt/anaconda3/envs/pyciemss/lib/python3.11',
 '/Users/mahs128/opt/anaconda3/envs/pyciemss/lib/python3.11/lib-dynload',
 '',
 '/Users/mahs128/opt/anaconda3/envs/pyciemss/lib/python3.11/site-packages',
 '/Users/mahs128/Repos/pyciemss/src',
 '/Users/mahs128/Repos/mira']

In [147]:
from mira.metamodel import * # ControlledConversion, NaturalConversion, Concept, Template
from mira.modeling.viz import GraphicalModel
from mira.modeling import *
from mira.modeling.askenet import * # 
import sympy

from pyciemss import interfaces
from pyciemss.PetriNetODE import interfaces

ModuleNotFoundError: No module named 'mira.modeling.askenet'

In [134]:
# Define concepts (model state variables)
susceptible = Concept(name='S', identifiers={'ido': '0000514'})
exposed = Concept(name='E', identifiers={'genepio': '0001538'})
symptomatic = Concept(name='I', identifiers={'ido': '0000512'})
asymptomatic = Concept(name='A', identifiers={'ido': '0000513'})
recovered = Concept(name='R', identifiers={'ido': '0000592'})
hospitalized = Concept(name='H', identifiers={'ido': '0000591'})
deceased = Concept(name='D', identifiers={'ncit': 'C28554'})



In [135]:
# Define conversion templates
beta, N, S, delta, I, A, ps, alpha, E, gamma, hosp, dnh, dh, los, H = sympy.symbols("beta, N, S, delta, I, A, ps, alpha, E, gamma, hosp, dnh, dh, los, H")

S_to_E = GroupedControlledConversion(
    controllers = [
        symptomatic, # Concept(name='I', identifiers={'ido': '0000512'}), 
        asymptomatic, # Concept(name='A', identifiers={'ido': '0000513'})
    ],
    subject=susceptible,
    outcome=exposed,
    rate_law=(beta/N)*S*(delta*I + A)
 )
E_to_I = NaturalConversion(
    subject=exposed,
    outcome=symptomatic,
    rate_law=(ps/alpha)*E
)
E_to_A = NaturalConversion(
    subject=exposed,
    outcome=asymptomatic,
    rate_law=((1-ps)/alpha)*E
)
I_to_R = NaturalConversion(
    subject=symptomatic,
    outcome=recovered,
    rate_law=gamma*(1-hosp-dnh)*I
)
I_to_H = NaturalConversion(
    subject=symptomatic,
    outcome=recovered,
    rate_law=gamma*(1-hosp-dnh)*I
)
I_to_D = NaturalConversion(
    subject=symptomatic,
    outcome=deceased,
    rate_law=gamma*dnh*I
)
A_to_R = NaturalConversion(
    subject=asymptomatic,
    outcome=recovered,
    rate_law=gamma*A
)
H_to_R = NaturalConversion(
    subject=hospitalized,
    outcome=recovered,
    rate_law=((1-dh)/los)*H
)
H_to_D = NaturalConversion(
    subject=hospitalized,
    outcome=deceased,
    rate_law=(dh/los)*H
)

In [136]:
# Define model template
tm = TemplateModel(
    templates=[S_to_E, E_to_I, E_to_A, I_to_R, I_to_H, I_to_D, A_to_R, H_to_R, H_to_D],
    parameters={
        'beta': Parameter(name='beta', value=0.55),
        'delta': Parameter(name='delta', value=1.5),
        'ps': Parameter(name='ps', value=0.7),
        'alpha': Parameter(name='alpha', value=4),
        'gamma': Parameter(name='gamma', value=0.2),
        'hosp': Parameter(name='hosp', value=0.1),
        'dnh': Parameter(name='dnh', value=0.001),
        'dh': Parameter(name='dh', value=0.1),
        'los': Parameter(name='los', value=7),
    }
)

In [137]:
type(tm)

mira.metamodel.template_model.TemplateModel

In [138]:
tm.dict()

{'templates': [{'rate_law': S*beta*(A + I*delta)/N,
   'type': 'GroupedControlledConversion',
   'controllers': [{'name': 'I',
     'identifiers': {'ido': '0000512'},
     'context': {}},
    {'name': 'A', 'identifiers': {'ido': '0000513'}, 'context': {}}],
   'subject': {'name': 'S', 'identifiers': {'ido': '0000514'}, 'context': {}},
   'outcome': {'name': 'E',
    'identifiers': {'genepio': '0001538'},
    'context': {}},
   'provenance': []},
  {'rate_law': E*ps/alpha,
   'type': 'NaturalConversion',
   'subject': {'name': 'E',
    'identifiers': {'genepio': '0001538'},
    'context': {}},
   'outcome': {'name': 'I', 'identifiers': {'ido': '0000512'}, 'context': {}},
   'provenance': []},
  {'rate_law': E*(1 - ps)/alpha,
   'type': 'NaturalConversion',
   'subject': {'name': 'E',
    'identifiers': {'genepio': '0001538'},
    'context': {}},
   'outcome': {'name': 'A', 'identifiers': {'ido': '0000513'}, 'context': {}},
   'provenance': []},
  {'rate_law': I*gamma*(-dnh - hosp + 1),


In [139]:
for k,v in tm:
    print(k)
tm.templates

templates
parameters
initials
observables
annotations


[GroupedControlledConversion(rate_law=S*beta*(A + I*delta)/N, type='GroupedControlledConversion', controllers=[Concept(name='I', identifiers={'ido': '0000512'}, context={}), Concept(name='A', identifiers={'ido': '0000513'}, context={})], subject=Concept(name='S', identifiers={'ido': '0000514'}, context={}), outcome=Concept(name='E', identifiers={'genepio': '0001538'}, context={}), provenance=[]),
 NaturalConversion(rate_law=E*ps/alpha, type='NaturalConversion', subject=Concept(name='E', identifiers={'genepio': '0001538'}, context={}), outcome=Concept(name='I', identifiers={'ido': '0000512'}, context={}), provenance=[]),
 NaturalConversion(rate_law=E*(1 - ps)/alpha, type='NaturalConversion', subject=Concept(name='E', identifiers={'genepio': '0001538'}, context={}), outcome=Concept(name='A', identifiers={'ido': '0000513'}, context={}), provenance=[]),
 NaturalConversion(rate_law=I*gamma*(-dnh - hosp + 1), type='NaturalConversion', subject=Concept(name='I', identifiers={'ido': '0000512'},

In [140]:
mira_m = Model(tm)

In [141]:
type(mira_m)

mira.modeling.Model

In [142]:
mira_m.variables

{('S', ('identity', 'ido:0000514')): <mira.modeling.Variable at 0x155334190>,
 ('E',
  ('identity', 'genepio:0001538')): <mira.modeling.Variable at 0x155314ad0>,
 ('I', ('identity', 'ido:0000512')): <mira.modeling.Variable at 0x155316f10>,
 ('A', ('identity', 'ido:0000513')): <mira.modeling.Variable at 0x155317b90>,
 ('R', ('identity', 'ido:0000592')): <mira.modeling.Variable at 0x155314690>,
 ('D', ('identity', 'ncit:C28554')): <mira.modeling.Variable at 0x155314050>,
 ('H', ('identity', 'ido:0000591')): <mira.modeling.Variable at 0x155316b10>}

In [146]:
# Convert the metamodel template model to petrinet model to take advantage of the to_json_file() function
from mira.modeling import petri
petri_m = petri.PetriNetModel(mira_m)

display(petri_m)
print(type(petri_m))

petri_m_fname = "Example_SEIARHD_model.json"
petri_m.to_json_file(fname = petri_m_fname)

<class 'mira.modeling.petri.PetriNetModel'>


In [144]:
# import petri_m the pyciemss way
the_model = interfaces.load_petri_model(petri_m)

AttributeError: 'PetriNetModel' object has no attribute 'parameters'

In [118]:
the_model

ScaledBetaNoisePetriNetODESystem(
	delta = Uniform(low: 1.350000023841858, high: 1.649999976158142),
	beta = Uniform(low: 0.4950000047683716, high: 0.6050000190734863),
	('S', 'E', ('I', 'A'), 'GroupedControlledConversion', 'rate') = Uniform(low: 0.0, high: 1.0),
	ps = Uniform(low: 0.6299999952316284, high: 0.7699999809265137),
	alpha = Uniform(low: 3.5999999046325684, high: 4.400000095367432),
	('E', 'I', 'NaturalConversion', 'rate') = Uniform(low: 0.0, high: 1.0),
	('E', 'A', 'NaturalConversion', 'rate') = Uniform(low: 0.0, high: 1.0),
	hosp = Uniform(low: 0.09000000357627869, high: 0.10999999940395355),
	gamma = Uniform(low: 0.18000000715255737, high: 0.2199999988079071),
	dnh = Uniform(low: 0.0008999999845400453, high: 0.0010999999940395355),
	('I', 'R', 'NaturalConversion', 'rate') = Uniform(low: 0.0, high: 1.0),
	('I', 'D', 'NaturalConversion', 'rate') = Uniform(low: 0.0, high: 1.0),
	los = Uniform(low: 6.300000190734863, high: 7.699999809265137),
	dh = Uniform(low: 0.09000000357

In [26]:
GraphicalModel.for_jupyter(tm, width="20%")

ValueError: Program dot not found in path.

In [27]:
tm2 = simplify_rate_laws(tm)
GraphicalModel.for_jupyter(tm2, width="20%")

ValueError: Program dot not found in path.

In [28]:
tm.dict()

{'templates': [{'rate_law': S*beta*(A + I*delta)/N,
   'type': 'GroupedControlledConversion',
   'controllers': [{'name': 'I', 'identifiers': {}, 'context': {}},
    {'name': 'A', 'identifiers': {}, 'context': {}}],
   'subject': {'name': 'S', 'identifiers': {}, 'context': {}},
   'outcome': {'name': 'E', 'identifiers': {}, 'context': {}},
   'provenance': []},
  {'rate_law': E*ps/alpha,
   'type': 'NaturalConversion',
   'subject': {'name': 'E', 'identifiers': {}, 'context': {}},
   'outcome': {'name': 'I', 'identifiers': {}, 'context': {}},
   'provenance': []},
  {'rate_law': E*(1 - ps)/alpha,
   'type': 'NaturalConversion',
   'subject': {'name': 'E', 'identifiers': {}, 'context': {}},
   'outcome': {'name': 'A', 'identifiers': {}, 'context': {}},
   'provenance': []},
  {'rate_law': I*gamma*(-dnh - hosp + 1),
   'type': 'NaturalConversion',
   'subject': {'name': 'I', 'identifiers': {}, 'context': {}},
   'outcome': {'name': 'R', 'identifiers': {}, 'context': {}},
   'provenance':

In [35]:
tm.parameters

{'beta': Parameter(name='beta', identifiers={}, context={}, value=0.55, distribution=None),
 'delta': Parameter(name='delta', identifiers={}, context={}, value=1.5, distribution=None),
 'ps': Parameter(name='ps', identifiers={}, context={}, value=0.7, distribution=None),
 'alpha': Parameter(name='alpha', identifiers={}, context={}, value=4.0, distribution=None),
 'gamma': Parameter(name='gamma', identifiers={}, context={}, value=0.2, distribution=None),
 'hosp': Parameter(name='hosp', identifiers={}, context={}, value=0.1, distribution=None),
 'dnh': Parameter(name='dnh', identifiers={}, context={}, value=0.001, distribution=None),
 'dh': Parameter(name='dh', identifiers={}, context={}, value=0.1, distribution=None),
 'los': Parameter(name='los', identifiers={}, context={}, value=7.0, distribution=None)}